In [ ]:
!pip install torchinfo > /dev/null
from torchinfo import summary

In [1]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as T
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torch.utils import data
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from skimage import io
from skimage import color
from skimage import transform
from skimage import util
import matplotlib.pyplot as plt

In [2]:
torch.__version__

'1.10.0+cu111'

In [2]:
import skimage

In [9]:
skimage.__version__

'0.18.3'

In [3]:
import tifffile

In [4]:
tifffile.__version__

'2021.11.2'

In [8]:
!python --version

Python 3.7.13


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
opt_dict = {
    'n_epochs': 200, 
    'batch_size': 4, 
    'lr': 2e-04, 
    'n_cpu': 8, 
    'latent_dim': 100, 
    'img_size': 224, 
    'channels': 4, 
    'n_critic': 5, 
    'clip_value': 0.01, 
    'sample_interval': 10
    }

class AttDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttDict, self).__init__(*args, **kwargs)
        self.__dict__=self

arg = AttDict(opt_dict)

In [ ]:
img_shape = (arg.channels, arg.img_size, arg.img_size)

cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if(torch.cuda.is_available()) else "cpu")

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_size, channels):

        super(Generator, self).__init__()
         # in: latent_size x 1 x 1

        self.gen = generator = nn.Sequential(

            nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),    
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(True),

            nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Conv2d(8, channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.Tanh()
            # out: 3 x 64 x 64
        )

    def forward(self, z):
        img = self.gen(z)
        return img


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels):
        super(Discriminator, self).__init__()

        self.discriminator = nn.Sequential(
            # in: 3 x 64 x 64

            nn.Conv2d(channels, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # out: 64 x 32 x 32

            nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            # out: 128 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # out: 256 x 8 x 8

            nn.Conv2d(128, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # out: 512 x 4 x 4

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, kernel_size=2, stride=2, padding=0, bias=False),
            # out: 1 x 1 x 1
            # Modification 1: remove sigmoid
            nn.Sigmoid()

            )


    def forward(self, img):
        output = self.discriminator(img)
        return output


In [ ]:
generator = Generator(arg.latent_dim, arg.channels)
discriminator = Discriminator(arg.channels)

In [ ]:
def weight_init(m):
    # weight_initialization: important for wgan
    #     class_name=m.__class__.__name__
    # if class_name.find('Conv')!=-1:
    #     m.weight.data.normal_(0,0.02)
    # elif class_name.find('Norm')!=-1:
    #     m.weight.data.normal_(1.0,0.02)
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    # classname=m.__class__.__name__
    # if classname.find('Conv')!=-1 or classname.find('Linear')!=-1:
    #     nn.init.kaiming_normal_(m.weight.data, a=0.0, mode='fan_in')
    # # init.normal_(m.weight.data, 0.0, 0.02)
    # if hasattr(m, 'bias') and m.bias is not None:
    #     nn.init.constant_(m.bias.data, 0.0)
    # elif classname.find('BatchNorm2d')!=-1:
    #     nn.init.normal_(m.weight.data, 0.0, 0.02)
    #     nn.init.constant_(m.bias.data, 0.0)

In [ ]:
discriminator.apply(weight_init)
generator.apply(weight_init)

Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(3

In [ ]:
# z = torch.randn(1, arg.latent_dim, 1, 1, device=device)
# fake = generator(z)
# fake = fake.detach()
# fake = fake[0].permute(1, 2, 0)
# plt.figure()
# plt.imshow((fake+1)/2)
# plt.show()

In [ ]:
summary(generator, input_size=(12, 100, 1, 1))

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                --                        --
├─Sequential: 1-1                        [12, 4, 224, 224]         --
│    └─ConvTranspose2d: 2-1              [12, 512, 4, 4]           819,200
│    └─BatchNorm2d: 2-2                  [12, 512, 4, 4]           1,024
│    └─ReLU: 2-3                         [12, 512, 4, 4]           --
│    └─ConvTranspose2d: 2-4              [12, 256, 8, 8]           2,097,152
│    └─BatchNorm2d: 2-5                  [12, 256, 8, 8]           512
│    └─ReLU: 2-6                         [12, 256, 8, 8]           --
│    └─ConvTranspose2d: 2-7              [12, 128, 14, 14]         131,072
│    └─BatchNorm2d: 2-8                  [12, 128, 14, 14]         256
│    └─ReLU: 2-9                         [12, 128, 14, 14]         --
│    └─ConvTranspose2d: 2-10             [12, 64, 28, 28]          131,072
│    └─BatchNorm2d: 2-11                 [12, 64, 28, 28] 

In [ ]:
summary(discriminator, input_size=(12, 4, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Discriminator                            --                        --
├─Sequential: 1-1                        [12, 1, 1, 1]             --
│    └─Conv2d: 2-1                       [12, 64, 112, 112]        4,096
│    └─LeakyReLU: 2-2                    [12, 64, 112, 112]        --
│    └─Conv2d: 2-3                       [12, 64, 56, 56]          65,536
│    └─BatchNorm2d: 2-4                  [12, 64, 56, 56]          128
│    └─LeakyReLU: 2-5                    [12, 64, 56, 56]          --
│    └─Conv2d: 2-6                       [12, 128, 28, 28]         131,072
│    └─BatchNorm2d: 2-7                  [12, 128, 28, 28]         256
│    └─LeakyReLU: 2-8                    [12, 128, 28, 28]         --
│    └─Conv2d: 2-9                       [12, 128, 14, 14]         262,144
│    └─BatchNorm2d: 2-10                 [12, 128, 14, 14]         256
│    └─LeakyReLU: 2-11                   [12, 128, 14, 14]       

In [ ]:
if cuda:
    generator.cuda()
    discriminator.cuda()

In [ ]:
transform_img = T.Compose([
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        T.RandomCrop((img_shape[1], img_shape[2]), padding=None)
])

In [ ]:
datapath = '/content/drive/MyDrive/BTPII'
ckpt = os.path.join(datapath, 'checkpoint')
if not os.path.exists(ckpt):
    os.mkdir(ckpt)

### List Dataset

In [ ]:
## data loader
class ListDataset(data.Dataset):

    def __init__(self, dataset, mode, crop_size, normalization='minmax', hidden_classes=None, overlap=False, use_dsm=False):

        # Initializing variables.
        self.root = os.path.join(datapath, dataset)
        self.dataset = dataset
        self.mode = mode
        self.crop_size = crop_size
        self.normalization = normalization
        self.hidden_classes = hidden_classes
        self.overlap = overlap
        self.use_dsm = use_dsm
        
        self.num_classes = 5 # For Vaihingen and Potsdam.
            
        if self.hidden_classes is not None:
            self.n_classes = self.num_classes - len(hidden_classes)
        else:
            self.n_classes = self.num_classes

        # Creating list of paths.
        self.imgs = self.make_dataset()

        # Check for consistency in list.
        if len(self.imgs) == 0:

            raise (RuntimeError('Found 0 images, please check the data set'))

    def make_dataset(self):

        # Making sure the mode is correct.
        assert self.mode in ['Train', 'Test', 'Val']

        # Setting string for the mode.
        img_dir = os.path.join(self.root, self.mode, 'Images')
        msk_dir = os.path.join(self.root, self.mode, 'Masks')
        if self.use_dsm:
            dsm_dir = os.path.join(self.root, self.mode, 'NDSM')

        # if self.mode == 'Val':
        #     img_dir = os.path.join(self.root, 'Train', 'Images')
        #     msk_dir = os.path.join(self.root, 'Train', 'Masks')
        #     if self.use_dsm:
        #         dsm_dir = os.path.join(self.root, 'Train', 'NDSM')

        data_list = sorted([f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))])

        # Creating list containing image and ground truth paths.
        items = []
        if self.dataset == 'Vaihingen':
            for it in data_list:
                item = (
                    os.path.join(img_dir, it),
                    os.path.join(msk_dir, it),
                    # os.path.join(dsm_dir, it.replace('top_mosaic_09cm_area', 'dsm_09cm_matching_area').replace('.tif', '_normalized.jpg'))
                )
                items.append(item)
        elif self.dataset == 'Potsdam':
            for it in data_list:
                if self.use_dsm:
                    item = (
                        os.path.join(img_dir, it),
                        os.path.join(msk_dir, it.replace('_IRRG.tif', '_label.tif')),
                        # os.path.join(msk_dir, it.replace('_IRRG.tif', '_label_noBoundary.tif')),
                        os.path.join(dsm_dir, it.replace('top_potsdam_', 'dsm_potsdam_').replace('_IRRG.tif', '_normalized_lastools.jpg'))
                    )
                else:
                    item = (
                        os.path.join(img_dir, it),
                        os.path.join(msk_dir, it.replace('_IRRG.tif', '_label.tif')),
                        # os.path.join(msk_dir, it.replace('_IRRG.tif', '_label_noBoundary.tif')),
                        # os.path.join(dsm_dir, it.replace('top_potsdam_', 'dsm_potsdam_').replace('_IRRG.tif', '_normalized_lastools.jpg'))
                    )
                items.append(item)
        
        # Returning list.
        return items
    
    def random_crops(self, img, msk, msk_true, n_crops):
        
        img_crop_list = []
        msk_crop_list = []
        msk_true_crop_list = []
        
        rand_fliplr = np.random.random() > 0.50
        rand_flipud = np.random.random() > 0.50
        rand_rotate = np.random.random()
        
        for i in range(n_crops):
            
            rand_y = np.random.randint(msk.shape[0] - self.crop_size[0])
            rand_x = np.random.randint(msk.shape[1] - self.crop_size[1])

            img_patch = img[rand_y:(rand_y + self.crop_size[0]),
                            rand_x:(rand_x + self.crop_size[1])]
            msk_patch = msk[rand_y:(rand_y + self.crop_size[0]),
                            rand_x:(rand_x + self.crop_size[1])]
            msk_true_patch = msk_true[rand_y:(rand_y + self.crop_size[0]),
                                      rand_x:(rand_x + self.crop_size[1])]
            
            if rand_fliplr:
                img_patch = np.fliplr(img_patch)
                msk_patch = np.fliplr(msk_patch)
                msk_true_patch = np.fliplr(msk_true_patch)
            if rand_flipud:
                img_patch = np.flipud(img_patch)
                msk_patch = np.flipud(msk_patch)
                msk_true_patch = np.flipud(msk_true_patch)
            
            if rand_rotate < 0.25:
                img_patch = transform.rotate(img_patch, 270, order=1, preserve_range=True)
                msk_patch = transform.rotate(msk_patch, 270, order=0, preserve_range=True)
                msk_true_patch = transform.rotate(msk_true_patch, 270, order=0, preserve_range=True)
            elif rand_rotate < 0.50:
                img_patch = transform.rotate(img_patch, 180, order=1, preserve_range=True)
                msk_patch = transform.rotate(msk_patch, 180, order=0, preserve_range=True)
                msk_true_patch = transform.rotate(msk_true_patch, 180, order=0, preserve_range=True)
            elif rand_rotate < 0.75:
                img_patch = transform.rotate(img_patch, 90, order=1, preserve_range=True)
                msk_patch = transform.rotate(msk_patch, 90, order=0, preserve_range=True)
                msk_true_patch = transform.rotate(msk_true_patch, 90, order=0, preserve_range=True)
                
            img_patch = img_patch.astype(np.float32)
            msk_patch = msk_patch.astype(np.int64)
            msk_true_patch = msk_true_patch.astype(np.int64)
            
            img_crop_list.append(img_patch)
            msk_crop_list.append(msk_patch)
            msk_true_crop_list.append(msk_true_patch)
        
        img = np.asarray(img_crop_list)
        msk = np.asarray(msk_crop_list)
        msk_true = np.asarray(msk_true_crop_list)
        
        return img, msk, msk_true
        
    def test_crops(self, img, msk, msk_true):
        
        n_channels = 3
        if self.use_dsm:
            n_channels = 4
        if self.overlap:
            w_img = util.view_as_windows(img,
                                         (self.crop_size[0], self.crop_size[1], n_channels),
                                         (self.crop_size[0] // 2, self.crop_size[1] // 2, n_channels)).squeeze()
            w_msk = util.view_as_windows(msk,
                                         (self.crop_size[0], self.crop_size[1]),
                                         (self.crop_size[0] // 2, self.crop_size[1] // 2))
            w_msk_true = util.view_as_windows(msk_true,
                                              (self.crop_size[0], self.crop_size[1]),
                                              (self.crop_size[0] // 2, self.crop_size[1] // 2))
        else:
            w_img = util.view_as_blocks(img, (self.crop_size[0], self.crop_size[1], n_channels)).squeeze()
            w_msk = util.view_as_blocks(msk, (self.crop_size[0], self.crop_size[1]))
            w_msk_true = util.view_as_blocks(msk_true, (self.crop_size[0], self.crop_size[1]))
        
        return w_img, w_msk, w_msk_true
        
    def shift_labels(self, msk):
        
        msk_true = np.copy(msk)
        
        cont = 0
        for h_c in self.hidden_classes:
            
            msk[msk == h_c - cont] = 100
            for c in range(h_c - cont + 1, self.num_classes):
                msk[msk == c] = c - 1
                msk_true[msk_true == c] = c - 1
            cont = cont + 1
        
        msk_true[msk == 100] = self.num_classes - len(self.hidden_classes)
        msk[msk == 100] = self.num_classes
        
        return msk, msk_true
    
    def mask_to_class(self, msk):
    
        msk = msk.astype(np.int64)
        new = np.zeros((msk.shape[0], msk.shape[1]), dtype=np.int64)
        
        msk = msk // 255
        msk = msk * (1, 7, 49)
        msk = msk.sum(axis=2)

        new[msk == 1 + 7 + 49] = 0 # Street.
        new[msk ==         49] = 1 # Building.
        new[msk ==     7 + 49] = 2 # Grass.
        new[msk ==     7     ] = 3 # Tree.
        new[msk == 1 + 7     ] = 4 # Car.
        new[msk == 1         ] = 5 # Surfaces.
        new[msk == 0         ] = 6 # Boundaries.

        return new
        
    def __getitem__(self, index):
        
        # Reading items from list.
        if self.use_dsm:
            img_path, msk_path, dsm_path = self.imgs[index]
        else:
            img_path, msk_path = self.imgs[index]
        
        # Reading images.
        img_raw = io.imread(img_path)
        msk_raw = io.imread(msk_path)
        if self.use_dsm:
            dsm_raw = io.imread(dsm_path)
            
        if len(img_raw.shape) == 2:
            img_raw = color.gray2rgb(img_raw)
        
        if self.use_dsm:
            img = np.full((img_raw.shape[0] + self.crop_size[0] - (img_raw.shape[0] % self.crop_size[0]),
                           img_raw.shape[1] + self.crop_size[1] - (img_raw.shape[1] % self.crop_size[1]),
                           img_raw.shape[2] + 1),
                          fill_value=0.0,
                          dtype=np.float32)
        else:
            img = np.full((img_raw.shape[0] + self.crop_size[0] - (img_raw.shape[0] % self.crop_size[0]),
                           img_raw.shape[1] + self.crop_size[1] - (img_raw.shape[1] % self.crop_size[1]),
                           img_raw.shape[2]),
                          fill_value=0.0,
                          dtype=np.float32)
        
        msk = np.full((msk_raw.shape[0] + self.crop_size[0] - (msk_raw.shape[0] % self.crop_size[0]),
                       msk_raw.shape[1] + self.crop_size[1] - (msk_raw.shape[1] % self.crop_size[1]),
                       msk_raw.shape[2]),
                      fill_value=0,
                      dtype=np.int64)
        
        img[:img_raw.shape[0], :img_raw.shape[1], :img_raw.shape[2]] = img_raw
        if self.use_dsm:
            img[:dsm_raw.shape[0], :dsm_raw.shape[1], -1] = dsm_raw
        msk[:msk_raw.shape[0], :msk_raw.shape[1]] = msk_raw
        
        msk = self.mask_to_class(msk)
        
        msk, msk_true = self.shift_labels(msk)
        
        # Normalization.
        img = (img / 255) - 0.5
        
        if self.mode == 'Train':
            
            img, msk, msk_true = self.random_crops(img, msk, msk_true, 3)
            
            img = np.transpose(img, (0, 3, 1, 2))
        
        elif self.mode == 'Val':
            
            img, msk, msk_true = self.test_crops(img, msk, msk_true)
            
            img = np.transpose(img, (0, 1, 4, 2, 3))
            msk = np.transpose(msk, (0, 1, 2, 3))
            msk_true = np.transpose(msk_true, (0, 1, 2, 3))

            # mini = np.random.randint(img.shape[0], size=10)
            mini_in = np.random.randint(img.shape[1], size=10)

            img = img[:10, mini_in, :, :, :]
            msk = msk[:10, mini_in, :, :]
            msk_true = msk_true[:10, mini_in, :, :]

        elif self.mode == 'Test':
            
            img, msk, msk_true = self.test_crops(img, msk, msk_true)

            img = np.transpose(img, (0, 1, 4, 2, 3))
            msk = np.transpose(msk, (0, 1, 2, 3))
            msk_true = np.transpose(msk_true, (0, 1, 2, 3))

            mini_in = np.random.randint(img.shape[1], size=10)

            img = img[:10, mini_in, :, :, :]
            msk = msk[:10, mini_in, :, :]
            msk_true = msk_true[:10, mini_in, :, :]
        
        msk[msk == self.num_classes + 1] = self.num_classes
        msk_true[msk_true == self.num_classes + 1] = self.num_classes

        # Splitting path.
        spl = img_path.split('/')

        # Turning to tensors.
        img = torch.from_numpy(img)
        msk = torch.from_numpy(msk)
        msk_true = torch.from_numpy(msk_true)

        # Returning to iterator.
        return img

    def __len__(self):

        return len(self.imgs)

### T

In [ ]:
trainDataset = ListDataset('Potsdam', 'Train', (img_shape[1], img_shape[2]), normalization='minmax', hidden_classes=[], overlap=False, use_dsm=True)
dataloader = DataLoader(trainDataset, batch_size=arg.batch_size, num_workers=1, shuffle=False)

In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=arg.lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=arg.lr, betas=(0.5, 0.999))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
if os.path.exists(os.path.join(ckpt, 'dcgan.pth')):
    state_dict = torch.load(os.path.join(ckpt, 'dcgan.pth'))
    generator.load_state_dict(state_dict['generator'])
    discriminator.load_state_dict(state_dict['discriminator'])
    #last 20

In [ ]:
criterion = nn.BCELoss()
fixed_noise = torch.randn(64, arg.latent_dim, 1, 1, device=device)
real_label = 1
fake_label = 0
# Stores generated images as training progresses.
img_list = []
# Stores generator losses during training.
G_losses = []
# Stores discriminator losses during training.
D_losses = []
iters = 0
last_epoch = 51


In [ ]:
print("Starting Training Loop...")
print("-"*25)
for epoch in range(last_epoch, arg.n_epochs):

    for i, imgs in enumerate(dataloader):

        # Configure input
        imgs = imgs.contiguous().view(imgs.shape[0]*3, arg.channels, arg.img_size, arg.img_size)
        real_imgs = Variable(imgs.type(Tensor))
        b_size = imgs.shape[0]

        discriminator.zero_grad()
        
        label = torch.full((b_size, ), real_label, device=device)
        output = discriminator(real_imgs).view(-1)
        errD_real = criterion(output.type(Tensor), label.type(Tensor))
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, arg.latent_dim, 1, 1, device=device)
        fake_data = generator(noise)
        label.fill_(fake_label)

        output = discriminator(fake_data.detach()).view(-1)
        errD_fake = criterion(output.type(Tensor), label.type(Tensor))
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake

        optimizer_D.step()

        generator.zero_grad()
        label.fill_(real_label)

        output = discriminator(fake_data).view(-1)
        errG = criterion(output.type(Tensor), label.type(Tensor))
        errG.backward()

        D_G_z2 = output.mean().item()
        optimizer_G.step()

        if i%5==0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, arg.n_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        G_losses.append(errG.item())
        D_losses.append(errD.item())

    if epoch % arg.sample_interval == 0:
        torch.save({
            'generator' : generator.state_dict(),
            'discriminator' : discriminator.state_dict(),
            'optimizerG' : optimizer_G.state_dict(),
            'optimizerD' : optimizer_D.state_dict()
            }, os.path.join(ckpt, 'dcgan.pth')) 


Starting Training Loop...
-------------------------
[21/200][0/7]	Loss_D: 1.3425	Loss_G: 10.6844	D(x): 0.9997	D(G(z)): 0.4086 / 0.0000
[21/200][5/7]	Loss_D: 0.0137	Loss_G: 7.5678	D(x): 0.9970	D(G(z)): 0.0105 / 0.0029
[22/200][0/7]	Loss_D: 0.0119	Loss_G: 6.7169	D(x): 0.9997	D(G(z)): 0.0115 / 0.0032
[22/200][5/7]	Loss_D: 0.0836	Loss_G: 11.5733	D(x): 0.9993	D(G(z)): 0.0771 / 0.0000
[23/200][0/7]	Loss_D: 0.0105	Loss_G: 10.8823	D(x): 0.9899	D(G(z)): 0.0002 / 0.0001
[23/200][5/7]	Loss_D: 0.2935	Loss_G: 14.6497	D(x): 0.9996	D(G(z)): 0.2416 / 0.0000
[24/200][0/7]	Loss_D: 0.0013	Loss_G: 12.9360	D(x): 0.9987	D(G(z)): 0.0000 / 0.0000
[24/200][5/7]	Loss_D: 0.0017	Loss_G: 9.5265	D(x): 0.9985	D(G(z)): 0.0002 / 0.0002
[25/200][0/7]	Loss_D: 0.0075	Loss_G: 7.3079	D(x): 0.9996	D(G(z)): 0.0069 / 0.0011
[25/200][5/7]	Loss_D: 0.0226	Loss_G: 7.7680	D(x): 0.9994	D(G(z)): 0.0216 / 0.0006
[26/200][0/7]	Loss_D: 0.0049	Loss_G: 8.0281	D(x): 0.9998	D(G(z)): 0.0047 / 0.0004
[26/200][5/7]	Loss_D: 0.1745	Loss_G: 12.4

KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()